In [8]:
import random


class CuatroEnLinea:
    def __init__(self, player1, player2,cant_filas,cant_columnas):

        self.player1, self.player2 = player1, player2
        self.player1_turn = random.choice([True, False])
        self.tablero = Tablero(cant_filas,cant_columnas)


    def play_game(self):
        #self.player1.start_game(1)
        #self.player2.start_game(2)

        terminamos = False

        print self.player1.simbolo
        print self.player2.simbolo
        
        while not terminamos:
            if self.player1_turn:
                player, simbolo = self.player1, self.player1.simbolo
            else:
                player, simbolo = self.player2, self.player2.simbolo

            self.mostrar_tablero()
            print "le toca al jugador " + str(simbolo)
            
            jugada_valida = False
            while not jugada_valida:
                accion = player.move(self)
                jugada_valida = accion in self.tablero.acciones_posibles()
                if not jugada_valida:
                    print str(accion) + " imposible.. volve a intentar"
                
                
            self.tablero.poner_ficha(accion,simbolo)      

            if self.gana_jugador(simbolo):
                print("ganó el jugador " + str(simbolo))
                self.mostrar_tablero()
                terminamos = True


            if self.hay_empate():
                print("tablero lleno")
                terminamos = True

            self.player1_turn = not self.player1_turn


            #if self.player_wins():
            #   player.reward(1, self.board)
            #  other_player.reward(-1, self.board)
            # break
            #if self.board_full(): # tie game
            #    player.reward(0.5, self.board)
            #    other_player.reward(0.5, self.board)
            #    break
            #other_player.reward(0, self.board)

    def gana_jugador(self,simbolo):
        return self.tablero.gana_jugador(simbolo)

    def mostrar_tablero(self):
        self.tablero.mostrar_tablero()

    def poner_ficha(self,columna,simbolo):
        self.tablero.poner_ficha(columna,simbolo)


    def hay_empate(self):
        return self.tablero.tablero_lleno()

    def dame_copia_del_tablero(self):
        return self.tablero.copy()


#########################################################################################################



class Tablero:

    
    def __init__(self, cant_filas,cant_columnas):

        self.cant_filas = cant_filas
        self.cant_columnas = cant_columnas
        self.tablero = [[0 for j in range(0,cant_columnas)] for i in range(0,cant_filas)]

    def __eq__(self,other):
        return self.tablero == other.tablero    
        
    def __hash__(self):
        res = []
        for i in self.tablero:
            res = res + i

        return hash(tuple(res))
        
    def copy(self):

        nuevo_tablero = Tablero(self.cant_filas,self.cant_columnas)
        nuevo_estado = nuevo_tablero.tablero

        for i in range(0,self.cant_filas):
            for j in range(0,self.cant_columnas):
                nuevo_estado[i][j] = self.tablero[i][j]

        return nuevo_tablero

    def tablero_lleno(self):
        acciones_posibles = self.acciones_posibles()
        return len(acciones_posibles) == 0

    def mostrar_tablero(self):
        for i in reversed(range(0,self.cant_filas)):
            print self.tablero[i]
            print "\n"


    def gana_jugador(self,simbolo):
        tablero = self.tablero
        for i in reversed(range(0,self.cant_filas)):
            for j in range(0,self.cant_columnas):
                if tablero[i][j] == simbolo:
                    if self.cuatro_en_linea(i,j,i-3,j) or self.cuatro_en_linea(i,j,i-3,j+3) or self.cuatro_en_linea(i,j,i,j+3) or self.cuatro_en_linea(i,j,i+3,j+3) or self.cuatro_en_linea(i,j,i+3,j) or self.cuatro_en_linea(i,j,i+3,j-3) or self.cuatro_en_linea(i,j,i,j-3) or self.cuatro_en_linea(i,j,i-3,j-3):
                        return True


        return False


    def hay_empate(self):
        return self.tablero_lleno() and not self.gana_jugador(1) and not self.gana_jugador(2)

    def build_range(self,i1,i2):

        rango = []
        if ( i1 < i2 ):
            rango = [i1,i1+1,i1+2,i1+3]
        elif ( i1 == i2 ):
            rango = [i1,i1,i1,i1]
        else:
            rango = [i1,i1-1,i1-2,i1-3]

        return rango


    def fila_disponible_en(self,j):
        for i in range(0,self.cant_filas):
            if self.tablero[i][j] == 0:
                return i
        return -1


    def poner_ficha(self,j,simbolo):
        tablero = self.tablero
        columna = j
        for i in range(0,self.cant_filas):
            if tablero[i][columna] == 0:
                tablero[i][columna] = simbolo
                break


    def cuatro_en_linea(self, i1,j1,i2,j2):

        tablero = self.tablero
        
        if i2 < 0 or i2 >= self.cant_filas or j2 < 0 or j2 >= self.cant_columnas:
            return False

        fila = self.build_range(i1,i2)
        columna = self.build_range(j1,j2)

        return (tablero[fila[0]][columna[0]] == tablero[fila[1]][columna[1]] == tablero[fila[2]][columna[2]] == tablero[fila[3]][columna[3]]) and tablero[fila[0]][columna[0]] != 0


    def proximo_movimiento_hay_ganador(self,simbolo):
        
        for accion in self.acciones_posibles():
            proximo_estado = self.proximo_estado_para(accion,simbolo)
            if self.gana_jugador(simbolo):
                return True
        
        return False    

    def acciones_posibles(self):
        accionesPosibles = []
        tablero = self.tablero
        for j in range(0,self.cant_columnas):
            termine_columna = False
            for i in range(0,self.cant_filas):
                if tablero[i][j] == 0 and not termine_columna:
                    accionesPosibles.append(j)
                    termine_columna = True


        return accionesPosibles


    def proximo_estado_para(self,accion,simbolo):
        nuevo_tablero = self.copy()
        nuevo_tablero.poner_ficha(accion,simbolo)

        return nuevo_tablero



#########################################################################################################


class Player():
    def __init__(self,simbolo):
        self.simbolo = simbolo
        self.breed = "human"

    def move(self,tablero):
        accion = int(raw_input("Qué columna elegís? "))
        
        return accion - 1


class RandomPLayer():
    def __init__(self,simbolo):
        self.breed = "random"
        self.simbolo = simbolo

    def move(self,tablero):
        return random.randint(0,6)

#########################################################################################################



class QPlayer():
    def __init__(self,simbolo,alpha,gamma,profundidad):
        self.breed = "qplayer"
        self.gamma = gamma
        self.alpha = alpha
        self.tabla_q = {}
        self.simbolo = simbolo
        self.profundidad = profundidad

    def move(self,juego):
        estado = juego.dame_copia_del_tablero()
        accion = self.mejor_accion_para(estado, self.profundidad)
        return accion


    def mejor_accion_para(self,estado,profundidad):
        acciones_posibles = estado.acciones_posibles()
        mejor_valor = -100
        mejor_accion = acciones_posibles[0]
        valor_candidato = 0
        
        for j in acciones_posibles:
            valor_candidato = self.q(estado,j,profundidad)
            if mejor_valor < valor_candidato:
                mejor_valor = valor_candidato
                mejor_accion = j

        viejo_valor = self.obtener_valor_en_tabla_q(estado,mejor_accion)
        self.tabla_q[(estado,mejor_accion)] = viejo_valor + self.alpha * ( self.recompensa() + self.gamma * mejor_valor - viejo_valor )

        return mejor_accion

    def q(self,estado,accion,profundidad):

        peor_valor = -100

        proximo_estado = estado.proximo_estado_para(accion,self.simbolo)

        
        if proximo_estado.gana_jugador(self.simbolo):
            peor_valor = 100
        elif proximo_estado.proximo_movimiento_hay_ganador(self.simbolo_del_rival()):
            peor_valor = -100
        elif proximo_estado.hay_empate():
            peor_valor = 0
        elif profundidad == 0:

            acciones_posibles_del_rival = proximo_estado.acciones_posibles()

            for j in acciones_posibles_del_rival:

                proximo_estado_a_explorar = proximo_estado.proximo_estado_para(j,self.simbolo)
                nuevas_acciones_posibles_mias = proximo_estado_a_explorar.acciones_posibles()

                for k in nuevas_acciones_posibles_mias:
                    valor_en_tabla_q = self.obtener_valor_en_tabla_q(proximo_estado_a_explorar,k)
                    if peor_valor > valor_en_tabla_q:
                        peor_valor = valor_en_tabla_q
        else:
            acciones_posibles_del_rival = proximo_estado.acciones_posibles()

            for j in acciones_posibles_del_rival:

                proximo_estado_a_explorar = proximo_estado.proximo_estado_para(j,self.simbolo)
                nuevas_acciones_posibles_mias = proximo_estado_a_explorar.acciones_posibles()

                for k in nuevas_acciones_posibles_mias:
                    valor_en_tabla_q = self.q(proximo_estado_a_explorar,k,profundidad-1)
                    if peor_valor > valor_en_tabla_q:
                        peor_valor = valor_en_tabla_q


        return peor_valor



    def recompensa(self):
        return 1

    def recompensa_por_defecto(self):
        return 1


    def obtener_valor_en_tabla_q(self,estado,accion):
        if (estado,accion) in self.tabla_q:
            return self.tabla_q.get((estado,accion))
        else:
            return 1

    def simbolo_del_rival(self):
        if self.simbolo == 1:
            return 2
        else:
            return 1


#########################################################################################################





In [9]:
#p1 = QPlayer(1,1,1,1)
p1 = Player(1)
p2=QPlayer(2,1,1,1)

#repetir 50 veces:
t = CuatroEnLinea(p1, p2,6,7)
t.play_game()

1
2
[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


le toca al jugador 1
QuÃ© columna elegÃ­s? 1
[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


le toca al jugador 2
[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[2, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


le toca al jugador 1
QuÃ© columna elegÃ­s? 1
[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


[2, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


le toca al jugador 2
[0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0]


[2, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


[2, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


le toca al jugador 1
QuÃ© columna elegÃ­s? 1
[0, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


[2, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0]


[2,